<a href="https://colab.research.google.com/github/hoanglam372000/Applied-Data-Science-Group-7/blob/nguyenthiman/8_12_TXL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1.1.1 Thư viện:

In [3]:
!git clone https://github.com/hoanglam372000/Applied-Data-Science-Group-7.git

fatal: destination path 'Applied-Data-Science-Group-7' already exists and is not an empty directory.


In [4]:
!pip install emoji
!pip install pyenchant
#!pip install pycontractions

In [5]:
import math
import pandas as pd
import nltk
import numpy as np
import emoji
#import contractions
import re
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import scipy.stats as st
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint
#print(stopwords.words('english'))
pd.set_option('display.max_colwidth', 200)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('names')
names = nltk.corpus.names.words('male.txt')+nltk.corpus.names.words('female.txt')
names_lower = [name.lower() for name in names]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [6]:
!pip install afinn

In [7]:
from afinn import Afinn
afinn = Afinn(language='en')

# TXL

In [8]:
# icon phổ biến
icon_df=pd.read_csv('/content/Applied-Data-Science-Group-7/icon.csv')
icon=icon_df['icon'].to_list()
# icon

In [18]:
stemmer = PorterStemmer()
# class xử lý vấn đề trùng ký tự trong từ, ví dụ yoooouuuu -> you, nooooo -> no.
#sử dụng pattern của regular expression, tuy nhiên phải quét thêm qua wordnet để tránh trường hợp như từ goose bị chuyển thành gose (mặc dù goose là từ đúng)
class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
    def replace(self, word):
        if wordnet.synsets(word):
            return word
        repl_word = self.repeat_regexp.sub(self.repl, word)
        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word
char_repeat_correct = RepeatReplacer()
def preprocessing_word(word):
  if word != emoji.demojize(word):
    return emoji.demojize(word)
  #word = contractions.fix(word)
  #word = stemmer.stem(word)
  temp=[ "<html_tag>",'<URL>','<HIDDEN>','<tag>',"<hash_tag>"]
  if word in temp:
    return word
  #số theo từ
  if word.isdigit():  
    return '<NUMBER>'
  try: 
    float(word)
    return '<NUMBER>'
  except:
    pass
  #word=word.lower() # viết thườngthường
  #spell_corrector = Speller(lang='en') #sửa chính tả theo từng từ 
  #word=spell_corrector(word)

  # rút gọn theo từ
  #lemmatizer = WordNetLemmatizer()
  #wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
  #pos_tagged = nltk.pos_tag(word.split())
  
  #word = [lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged]
  #word = ' '.join(word)

  #word=word.translate(str.maketrans('', '', string.punctuation)) # chấm câu theo từ (vì tránh emoij)
  if word in icon:
    return word
  else:
    for icon_ in icon:
      if icon_ in word:
        return icon_
  word = re.sub(pattern=r'[!"#$%&\'()*+,-\./:;<=>?@\[\]^_`{|}~0-9ï¿½]+', repl='', string=word)
  word = char_repeat_correct.replace(word)
  word = stemmer.stem(word)
  if (word in  names):
    return '<Name>'
  #word = stemmer.stem(word)
  word = word.lower()
  
  if word==''or word ==' ':
    return "<Special_Character>"
  
  if 'not' not in word and len(word.split())!=1:
    return word.split()[0]+'_'
  return word.replace(' ','_')

def preprocessing_string(text):
   # emoij
   raw_text=emoji.demojize(text)
   e_r=r':\w+_\w+:'
   emoij_list=re.findall(e_r,raw_text)
   for e in emoij_list:
     raw_text=raw_text.replace(e,' ' + emoji.emojize(e) + ' ') # "hi❤️👍"

   #parser = BeautifulSoup(raw_text, "html.parser")
   #raw_text = parser.get_text(separator = "<html_tag>")
   raw_text = re.sub(pattern=r'https?://\S+|www\.\S+', repl=' <URL> ', string=raw_text)
   raw_text = re.sub(pattern='\*{3,}?', repl=' <HIDDEN> ', string=raw_text)
   raw_text = re.sub(pattern=r'@\w+', repl=' <tag> ', string=raw_text)
   raw_text = re.sub(pattern=r'#\w+', repl=' <hash_tag> ', string=raw_text)
  
   s_=r"[#!\"$%&\'*+,-\.?@\[\]():;/`{|}]+"
   for i in re.findall(s_,raw_text):
     raw_text=raw_text.replace(i, ' '+i+' ')
   new_tweet = raw_text.split()

   word_list=raw_text.split()
   word_list_=[]
   for i in range(len(word_list)):
     word_list[i]=preprocessing_word(word_list[i])
     if word_list[i]!="<Special_Character>":
       word_list_.append(word_list[i])
   if len(word_list)!=1:
     word_list=word_list_
   return word_list
  
def preprocessing_text(tweet):
   raw_text = str(tweet).strip().replace('\n', ' ')#.lower()
   new_tweet = raw_text.split()
   len_=len(new_tweet)
   index2word = { w:new_tweet[w] for w in range(len_)}
   word_list=raw_text.split()
   return index2word,dict(zip([*range(len_)],map(preprocessing_string,word_list)))

In [19]:
# Test:
tweet = ' My:)) mom just texted.......me and told *ship* James that Rodney3 charlie hi❤️👍 wasï¿½ *** chasing?? and fireflies... in their :)) backyard. Awwwww I`m miss him!!! @hoanglam 123 ://    http://twitpic.com/66pn1'
tweet2 = 'nooooooooo'
index2word_temp,preprocessed_tweet_temp = preprocessing_text(tweet)
print('index2word: ',index2word_temp)
print('preprocessed_tweet: ',preprocessed_tweet_temp)

index2word:  {0: 'My:))', 1: 'mom', 2: 'just', 3: 'texted.......me', 4: 'and', 5: 'told', 6: '*ship*', 7: 'James', 8: 'that', 9: 'Rodney3', 10: 'charlie', 11: 'hi❤️👍', 12: 'wasï¿½', 13: '***', 14: 'chasing??', 15: 'and', 16: 'fireflies...', 17: 'in', 18: 'their', 19: ':))', 20: 'backyard.', 21: 'Awwwww', 22: 'I`m', 23: 'miss', 24: 'him!!!', 25: '@hoanglam', 26: '123', 27: '://', 28: 'http://twitpic.com/66pn1'}
preprocessed_tweet:  {0: ['my', ':))'], 1: ['mom'], 2: ['just'], 3: ['text', 'me'], 4: ['and'], 5: ['told'], 6: ['ship'], 7: ['jame'], 8: ['that'], 9: ['rodney'], 10: ['charli'], 11: ['hi', ':red_heart:', ':thumbs_up:'], 12: ['wa'], 13: ['<HIDDEN>'], 14: ['chase'], 15: ['and'], 16: ['firefli'], 17: ['in'], 18: ['their'], 19: [':))'], 20: ['backyard'], 21: ['aw'], 22: ['i', 'm'], 23: ['miss'], 24: ['him'], 25: ['<tag>'], 26: ['<NUMBER>'], 27: [':/'], 28: ['<URL>']}
